In [1]:
import os.path
import cv2
import pickle
import numpy as np
import json
import imageio
from flask import Flask, Response, jsonify, make_response, request
from face_utils import encode_faces, load_model, recong_face_c, generate_response, search_face_by_distance

e:\python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
global model, sess, graph, embeddings
embedding_dat_path = './embedding.dat'
embeddings = {}
model_path = '../models/20170511-185253'
classifier_filename = './my_classifier.pkl'
if os.path.exists(embedding_dat_path):
    with open(embedding_dat_path, 'rb') as infile:
        embeddings = pickle.load(infile)
model, sess, graph, pnet, rnet, onet = load_model(model_path, classifier_filename)

Creating networks and loading parameters
Loading feature extraction model
Model directory: ../models/20170511-185253
Metagraph file: model-20170511-185253.meta
Checkpoint file: model-20170511-185253.ckpt-80000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from ../models/20170511-185253\model-20170511-185253.ckpt-80000
load classifier file-> ./my_classifier.pkl


In [3]:
classifier_filename = 'my_classifier.pkl'
classifier_filename_exp = os.path.expanduser(classifier_filename)
if os.path.exists(classifier_filename_exp):
    with open(classifier_filename_exp, 'rb') as infile:
        model = pickle.load(infile)
        print('load classifier file-> %s' % classifier_filename_exp)

load classifier file-> my_classifier.pkl


In [4]:
def register(id, img_file):
    img = imageio.imread(img_file)
    embeddings_boxes = encode_faces(graph, sess, pnet, rnet, onet, img)
    if len(embeddings_boxes) == 1:
        if id not in embeddings:
            embeddings[id] = []
        embeddings[id].append(embeddings_boxes[0][0])
        
def batch_reg(folder):
    for id in os.listdir(folder):
        for img in os.listdir(os.path.join(folder, id)):
            try:
                register(id, os.path.join(folder, id, img))
            except:
                print('skip' + os.path.join(folder, id, img))

In [25]:
batch_reg('./median')
# register(id, './out_dir/Abdullah_Gul/Abdullah_Gul_0002.png')

skip./median\Abdullah_Gul\Abdullah_Gul_0002.png
skip./median\Abdullah_Gul\Abdullah_Gul_0008.png
skip./median\Abdullah_Gul\Abdullah_Gul_0010.png
skip./median\Abdullah_Gul\Abdullah_Gul_0013.png
skip./median\Abdullah_Gul\Abdullah_Gul_0014.png
skip./median\Bill_Gates\Bill_Gates_0008.png
skip./median\George_W_Bush\George_W_Bush_0004.png
skip./median\George_W_Bush\George_W_Bush_0005.png
skip./median\George_W_Bush\George_W_Bush_0011.png
skip./median\George_W_Bush\George_W_Bush_0063.png
skip./median\George_W_Bush\George_W_Bush_0073.png
skip./median\George_W_Bush\George_W_Bush_0079.png
skip./median\George_W_Bush\George_W_Bush_0099.png
skip./median\George_W_Bush\George_W_Bush_0101.png
skip./median\George_W_Bush\George_W_Bush_0116.png
skip./median\George_W_Bush\George_W_Bush_0117.png
skip./median\George_W_Bush\George_W_Bush_0148.png
skip./median\George_W_Bush\George_W_Bush_0159.png
skip./median\George_W_Bush\George_W_Bush_0163.png
skip./median\George_W_Bush\George_W_Bush_0170.png
skip./median\Geo

In [27]:
# %load_ext autoreload
# %autoreload 2
# print (len(embeddings))
import make_classifier
import imp  
imp.reload(make_classifier)  
model = make_classifier.make_classifier(sess, graph, embeddings, 'my_classifier.pkl', kernal='linear', probability=True)

make_classifier with
Training classifier
Saved classifier model to file "my_classifier.pkl"


In [33]:
# print(model.decision_function_shape)
# print(vars(model))
# print(model.support_vectors_.shape[1])

print (len(embeddings['gai']))

2


In [46]:
# img = imageio.imread('./out_dir/George_W_Bush/George_W_Bush_0005.png')
img = imageio.imread('gwb.jpg')
embeddings_boxes = encode_faces(graph, sess, pnet, rnet, onet, img)
emb = embeddings_boxes[0][0]
dfs = model.predict_proba(emb.reshape(1,128))
print(dfs)
print(model.classes_)
min_idx = np.argmax(dfs, axis=1)
print(model.classes_[min_idx])
print(dfs[0][min_idx])

[[0.04063589 0.07709223 0.42640071 0.06115703 0.04617131 0.05696048
  0.05099897 0.03788681 0.04292298 0.0285804  0.04523762 0.01050571
  0.03196609 0.00870397 0.02022939 0.01455041]]
['Abdullah_Gul' 'Bill_Gates' 'George_W_Bush' 'Gray_Davis'
 'Hillary_Clinton' 'James_Blake' 'Jiang_Zemin' 'John_Allen_Muhammad'
 'Kim_Clijsters' 'Nicanor_Duarte_Frutos' 'fbb' 'gai' 'ljl' 'pgone' 'zw'
 'zzy']
['George_W_Bush']
[0.42640071]
